In [1]:
import pandas as pd
from fredapi import Fred
fred = Fred(api_key='e7d647d3a626afa2df49109657f48e3e')

In [2]:
bitcoinprice = fred.get_series('CBBTCUSD')
goldprice = fred.get_series('ID7108')
market = fred.get_series('SP500')
riskfreerate = fred.get_series('TB3MS')

In [3]:
df = pd.concat([bitcoinprice,goldprice,market,riskfreerate], axis=1)

In [4]:
df.columns = ('bitcoinprice','goldprice','market','riskfreerate')

In [5]:
df = df.loc['2015-06-01':'2020-12-20']

In [6]:
df = df.fillna(method='ffill')

In [7]:
df = df[df.index.day==1]

In [8]:
df['bt-1'] = df['bitcoinprice'].shift(1)
df['gp-1'] = df['goldprice'].shift(1)
df['market-1'] = df['market'].shift(1)

In [9]:
import numpy as np

In [10]:
df['log_b'] = np.log(df['bitcoinprice'])/df['bitcoinprice'].shift(1)
df['log_g'] = np.log(df['goldprice'])/df['goldprice'].shift(1)
df['log_m'] = np.log(df['market'])/df['market'].shift(1)
df['rfr-1'] = df['riskfreerate']/1200

In [11]:
rbt = df["log_b"]
rgt = df["log_g"]
rmt = df["log_b"]
rft = df["rfr-1"]

In [12]:
#Used for calculating correlations
corr1 = rbt.corr(rgt)
corr2 = rbt.corr(rmt)

In [13]:
corr1

0.5825614763054296

In [14]:
corr2

1.0

In [15]:
#Dropping the index for calculating regression
df=df.drop(df.index[0])

In [16]:
rbt = df["log_b"]
rgt = df["log_g"]
rmt = df["log_b"]
rft = df["rfr-1"]

In [17]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
x = rbt - rft
y = rgt - rft
z = rmt - rft

In [20]:
y = y.values.reshape(-1, 1)
x = x.values.reshape(-1, 1)
z = z.values.reshape(-1, 1)

In [21]:
lr=LinearRegression()

In [22]:
#Regress x on z
lr.fit(x,z)
print(lr.intercept_, lr.coef_)
#In this case α = 2.60208521e-18 and β = 1, This means that as the x increases by 1%, then the value of bitcoin will increase by 2.6021 

[2.60208521e-18] [[1.]]


In [23]:
#Regress y on z
lr.fit(y,z)
print(lr.intercept_, lr.coef_)
#In this case α = -0.04016107 and β = 3.7502841, This means that as the x increases by 1%, then the value of goldprice will decrease by 3.7503

[-0.04016107] [[3.7502841]]
